In [1]:
from time import time
import numpy
import math

In [2]:
class State:
    def __init__(self):
        self.__values = []
    
    def setValues(self, values):
        self.__values = values[:]
    
    def getValues(self):
        return self.__values[:]
    
    def __str__(self):
        return str(self.__values)
    
    def __repr__(self):
        return repr(self.__values)

In [3]:
class Problem:
    def __init__(self, n, values):
        self.__n = n
        self.__initialState = State()
        self.__initialState.setValues(values)
        self.__finalState = State()
    
    def expand(self,state):
        children = []
        values = state.getValues()
        
        for i in range(0, len(values)):
            for j in range(0, len(values)):
                if values[i][j] == 0:
                    childValues = numpy.copy(values)
                    childValues[i][j] = 1
                    
                    child = State()
                    child.setValues(childValues)
                    
                    children.append(child)
        return children
    
    def heuristic(self, state):
        values = state.getValues()
        boardSize = self.__n
        
        # Check if 2 queens are on the same row / col
        for i in range(0, boardSize):
            if sum(values[i]) > 1 or sum(values[:,i]) > 1:
                return math.inf
        
        numberOfQueens = sum(numpy.sum(values, axis=0))
        if numberOfQueens <= boardSize:
            return boardSize - numberOfQueens
        
        return math.inf
    
    
    def readFromFile(self, fileName):
        values = numpy.loadtxt(fileName, unpacking = True, dtype = numpy.int8)
        self.__initialState.setValues(values)
    
    def getInitialState(self):
        return self.__initialState
    
    def setInitialState(self, initialState):
        self.__initialState = initialState
    
    def setFinalState(self, finalState):
        self.__finalState = finalState
    
    def getFinalState(self):
        return self.__finalState
    
    def getSize(self):
        return self.__n
    

In [4]:
class Controller:
    def __init__(self, problem):
        self.__problem = problem
    
    def orderStates(self, listOfStates):
        return sorted(listOfStates, key=self.__problem.heuristic)
    
    def bfs(self):
        initialState = self.__problem.getInitialState()
        nodesToVisit = [initialState]
        
        while len(nodesToVisit) != 0:
            currentState = nodesToVisit.pop(0)
            
            if self.isSolution(currentState):
                return currentState

            currentStateChildren = self.__problem.expand(currentState)
            nodesToVisit = nodesToVisit + currentStateChildren
            
        return State()
    
    def cutSearch(self, state):
        values = state.getValues()
        boardSize = self.__problem.getSize()

        if sum(numpy.sum(values, axis = 1)) >= boardSize:
            return True

        return False

    def dfs(self):
        initialState = self.__problem.getInitialState()
        nodesToVisit = [initialState]
        
        while len(nodesToVisit) != 0:
            currentState = nodesToVisit.pop(0)
            
            if self.isSolution(currentState):
                return currentState
            
            currentStateChildren = []
            
            if not self.cutSearch(currentState):
                currentStateChildren = self.__problem.expand(currentState)
                
            nodesToVisit = currentStateChildren + nodesToVisit
        
        return State()
        
    
    def gbfs(self):
        initialState = self.__problem.getInitialState()
        nodesToVisit = [initialState]
        
        while len(nodesToVisit) != 0:
            currentState = nodesToVisit.pop(0)
            
            if self.isSolution(currentState):
                return currentState

            currentStateChildren = self.__problem.expand(currentState)
            nodesToVisit = self.orderStates(nodesToVisit + currentStateChildren)
            
        return State()
        
    def isSolution(self, state):
        values = state.getValues()
        boardSize = self.__problem.getSize()
        
        # Check rows and columns
        for i in range(0, boardSize):
            if sum(values[i]) != 1 or sum(values[:,i]) != 1:
                return False
            
        # Check diagonals
        for row in range(0, boardSize):
            for col in range(0, boardSize):
                if values[row, col] == 1:
                    #Check right diagonal
                    i = row + 1
                    j = col + 1
                    while(i < boardSize and j < boardSize):
                        if values[i,j] == 1:
                            return False
                        i += 1
                        j += 1
                    
                    #Check left diagonal
                    i = row + 1
                    j = col - 1
                    while(i < boardSize and j >= 0):
                        if values[i,j] == 1:
                            return False
                        i += 1
                        j -= 1
                    
        return True

In [5]:
class UI:
    def __init__(self):
        pass
    
    def printMainMenu(self):
        menu = "0.Exit\n"
        menu += "1.DFS\n"
        menu += "2.BFS\n"
        menu += "3.GBFS\n"
        
        print(menu)

    def run(self):
        while True:
            n = int(input("Enter problem size:"))
            p = Problem(n, numpy.zeros((n,n), numpy.int8))
            c = Controller(p)
            
            self.printMainMenu()

            command = int(input("Enter finding method:"))
            if command == 0:
                return
            elif command == 1:
                print("-------------------------------------DFS-------------------------------------\n")
                print("Solution found:\n", c.dfs())
            elif command == 2:
                print("-------------------------------------BFS-------------------------------------\n")
                print("Solution found:\n", c.bfs())
            elif command == 3:
                print("-------------------------------------GBFS-------------------------------------\n")
                print("Solution found:\n", c.gbfs())

In [6]:
n = 5
p = Problem(n, numpy.zeros((n,n), numpy.int8))
c = Controller(p)

print(c.dfs())

[[1 0 0 0 0]
 [0 0 1 0 0]
 [0 0 0 0 1]
 [0 1 0 0 0]
 [0 0 0 1 0]]


In [ ]:
n = 4
p = Problem(n, numpy.zeros((n,n), numpy.int8))
c = Controller(p)

print(c.bfs())

In [8]:
n = 5
p = Problem(n, numpy.zeros((n,n), numpy.int8))
c = Controller(p)

print(str(c.gbfs()))

[[1 0 0 0 0]
 [0 0 1 0 0]
 [0 0 0 0 1]
 [0 1 0 0 0]
 [0 0 0 1 0]]


In [ ]:
ui = UI()
ui.run()

Enter problem size: 4


0.Exit
1.DFS
2.BFS
3.GBFS



Enter finding method: 2


-------------------------------------BFS-------------------------------------

Solution found:
 [[0 1 0 0]
 [0 0 0 1]
 [1 0 0 0]
 [0 0 1 0]]


Enter problem size: 5


...

-------------------------------------GBFS-------------------------------------

Solution found:
 [[1 0 0 0 0]
 [0 0 1 0 0]
 [0 0 0 0 1]
 [0 1 0 0 0]
 [0 0 0 1 0]]


Enter problem size: 6


0.Exit
1.DFS
2.BFS
3.GBFS



Enter finding method: 3


-------------------------------------GBFS-------------------------------------

